In [1]:
import dataset

train_loader_dict, test_loader = dataset.make_train_test_datasets(dataset="mnist", num_labels=10, 
                                alpha_dirichlet= 10, num_nodes=16, train_batch=64, test_batch=100)

def count_elements(dataloader):
    count = 0
    for X, y in dataloader:
        count += len(y)
    return count

for i in range(len(train_loader_dict)):
    print(count_elements(train_loader_dict[i]))

5.6%

100.0%


Extracting /root/CD-SGD/datasets/cache/MNIST/raw/train-images-idx3-ubyte.gz to /root/CD-SGD/datasets/cache/MNIST/raw


100.0%


Extracting /root/CD-SGD/datasets/cache/MNIST/raw/train-labels-idx1-ubyte.gz to /root/CD-SGD/datasets/cache/MNIST/raw



100.0%


Extracting /root/CD-SGD/datasets/cache/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/CD-SGD/datasets/cache/MNIST/raw



100.0%


Extracting /root/CD-SGD/datasets/cache/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/CD-SGD/datasets/cache/MNIST/raw

Proportions for label 1 [0.0674734  0.11258481 0.06823908 0.04459326 0.10154952 0.0493773
 0.05929911 0.0616702  0.05761078 0.05428442 0.03679206 0.04187446
 0.05939473 0.05857774 0.06872021 0.05795893]
3600
4015
3927
3409
3854
3988
3414
3658
2993
3997
3100
3591
3475
4510
4460
3930


In [4]:
train_loader_dict, test_loader = dataset.make_train_test_datasets(dataset="mnist", num_labels=10, 
                                alpha_dirichlet= 0.1, num_nodes=16, train_batch=64, test_batch=100)
for i in range(len(train_loader_dict)):
    print(count_elements(train_loader_dict[i]))

Proportions for label 1 [1.44881783e-10 2.77119235e-02 1.90713379e-03 5.43181477e-02
 1.90199221e-01 2.21833325e-03 6.22535454e-01 6.37615414e-06
 1.98624734e-03 1.47353540e-02 9.37357500e-04 1.79410486e-06
 8.34425973e-02 1.20608356e-09 2.00611443e-09 5.77519711e-08]
9079
8729
468
4820
1175
2675
6997
880
3112
10303
1072
3437
875
5801
87
446


In [7]:
train_loader_dict, test_loader = dataset.make_train_test_datasets(dataset="mnist", num_labels=10, 
                                alpha_dirichlet= 1000, num_nodes=16, train_batch=64, test_batch=100)
s = 0
for i in range(len(train_loader_dict)):
    c = count_elements(train_loader_dict[i])
    print(c)
    s += c
print(s)

Proportions for label 1 [0.06156412 0.06446907 0.06303828 0.06425248 0.0618228  0.0613195
 0.06404928 0.06132321 0.06537084 0.05890092 0.06317416 0.06259525
 0.06039699 0.06093302 0.06387691 0.06291317]
3723
3734
3774
3730
3758
3760
3833
3702
3766
3710
3723
3731
3691
3752
3758
3773
59918


In [6]:
from utils import topology, dp_account
import numpy as np
import misc, torch

num_nodes = 16
sigma_cor = 1

W = topology.FixedMixingMatrix(topology_name= "grid", n_nodes= 16)(0)
adjacency_matrix = np.array(W != 0, dtype=float)
adjacency_matrix = adjacency_matrix - np.diag(np.diag(adjacency_matrix))
degree_matrix = np.diag(adjacency_matrix @ np.ones_like(adjacency_matrix[0]))

W = torch.tensor(W, dtype= torch.float)
# Noise tensor: shape (num_nodes, num_nodes, model_size)
V = torch.randn(num_nodes, num_nodes, 3) # distribution N(0, 1)
V.mul_(sigma_cor) # rescaling ==> distribution N (0, sigma_cor^2)

# Antisymmetry property and neighbours
V = misc.to_antisymmetric(V, W)
print(V[0])


tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.2553, -0.0380, -0.2279],
        [ 0.0000, -0.0000,  0.0000],
        [-0.2513, -0.0448,  0.1771],
        [ 0.2182, -0.2342,  0.2579],
        [-0.0000, -0.0000, -0.0000],
        [ 0.0000,  0.0000, -0.0000],
        [ 0.0000, -0.0000, -0.0000],
        [ 0.0000, -0.0000,  0.0000],
        [-0.0000,  0.0000, -0.0000],
        [-0.0000, -0.0000,  0.0000],
        [ 0.0000, -0.0000,  0.0000],
        [-0.0236, -0.0547,  0.1812],
        [-0.0000, -0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000]])


In [12]:
# Test
for i in range(num_nodes):
    # diag part
    if not torch.all(V[i, i].eq(0)):
        print("We have a non null part of diagonal")
    for j in range(num_nodes):
        # antisymmetric part 
        if not torch.equal(V[i, j], - V[j, i]):
            print("not antisymmetric")
        # neighbors part
        if W[i, j] == 0:
            if not torch.all(V[i, j].eq(0)):
                print("neighbours part not verified")
print("Cool")

Cool
